<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/043_AgentLanguage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🧠 How Your Agent Communicates with the LLM

## 🧭 The Role of `AgentLanguage`

The `AgentLanguage` component acts as a **translator** between your agent’s structured logic (Goals, Actions, Memory, Environment – the GAME framework) and the **LLM’s input/output format** (text or function calling).

Think of it like a diplomatic interpreter between your agent and the model.

### Responsibilities

1. **Prompt Construction**

   * Builds the message payload to send to the LLM using:

     * Goals
     * Available Actions
     * Memory
     * Environment Info

2. **Response Parsing**

   * Interprets the LLM’s response and determines which action (tool) the agent should take next.

---

## 🔄 Where It Fits in the Agent Loop

The `AgentLanguage` is used at two key stages:

```python
# 1. Construct prompt
prompt = self.construct_prompt(self.goals, memory, self.actions)

# 2. Send to LLM
response = self.prompt_llm_for_action(prompt)

# 3. Parse LLM response into an action + arguments
action, invocation = self.get_action(response)
```

---

## 🧰 Example AgentLanguage Implementations

### 1. **AgentJsonActionLanguage**

* Uses markdown blocks like \`\`\`action to specify tool usage.
* Supports reasoning + action in the same message.
* Example: Let me analyze the file.

  ```action
  {
    "tool": "read_file",
    "args": {
      "file_path": "file.txt"
    }
  }

  ```
* parse_response` extracts the JSON block and parses it into `{ tool, args }.

### 2. **AgentFunctionCallingActionLanguage**

* Uses OpenAI-style tool/function calling (structured, no parsing).
* No inline reasoning text.
* Simpler + less error-prone.
* Ideal for production.

---

## 🔄 Swappable Languages = Flexibility

You can switch out the `AgentLanguage` without changing agent logic:

```python
agent = Agent(
    goals=goals,
    agent_language=AgentFunctionCallingActionLanguage(),  # or AgentJsonActionLanguage()
    action_registry=registry,
    generate_response=llm.generate,
    environment=env
)
```

✅ This lets you:

* Experiment with different prompting strategies
* Adapt to different LLM providers
* Improve or debug communication without modifying core logic

---

## 🎯 Why AgentLanguage Matters

| Benefit                  | Description                               |
| ------------------------ | ----------------------------------------- |
| Centralized Logic        | One place to handle prompting and parsing |
| Experimental Flexibility | Easily A/B test formats or models         |
| Robustness               | Structured error handling and retry logic |
| Future-Proof             | Swap formats as LLMs evolve               |





## 🤖 What is `AgentLanguage`?

**`AgentLanguage` is a class that defines how your agent "talks" to the LLM.**

Think of it like a **communication protocol or adapter**. Your agent has structured internal logic — goals, tools (actions), memory — but the **LLM only understands prompts and returns text or structured outputs**.

So `AgentLanguage` does two things:

---

### 🛠 1. It builds the prompt to send to the LLM

* Based on:

  * 🧠 Memory (past messages)
  * 🎯 Goals
  * 🧰 Tools / Actions
  * 🌍 Environment info

> "Hey LLM, here’s what I’m trying to do, what I can do, and what I’ve done so far. What should I do next?"

It formats that into a message or API call (depending on which `AgentLanguage` subclass you're using).

---

### 📬 2. It parses the LLM's response

* If the LLM returns:

  * A **tool call** (function to run + arguments), `AgentLanguage` extracts that.
  * A **text message**, it captures that and returns it.

This parsed response is what tells the agent:

> "The LLM wants me to run this action next, with these arguments."

---

## 🧪 There are Multiple AgentLanguage Styles

| Class                                | What it does                                   | Output style                                  |
| ------------------------------------ | ---------------------------------------------- | --------------------------------------------- |
| `AgentFunctionCallingActionLanguage` | Uses OpenAI’s native tool-calling API          | Structured function calls (no parsing needed) |
| `AgentJsonActionLanguage`            | Uses markdown with JSON blocks (` ```action `) | Text + JSON block (custom parsing required)   |

You choose the one that fits your needs — structured precision or flexible reasoning.

---

## 📦 Analogy: `AgentLanguage` is Like...

> A **translator** for your structured agent talking to a text-based LLM.
> Without it, your agent and the model would be speaking two different languages.




### ✅ `AgentLanguage` is **a Python class** — part of the custom framework you’ve been working with in these lectures.

---

### 📦 So what *is* it, really?

Think of `AgentLanguage` like this:

* It’s a **class** (likely an abstract base class or interface) in your agent framework.
* You don’t `pip install agentlanguage` — it’s part of your project (or the course framework).
* You or the framework author can write different **subclasses** of `AgentLanguage` to change how prompts and responses are handled.

---

### 📚 Example

Here’s what it might look like under the hood:

```python
# agent_language.py
class AgentLanguage:
    def construct_prompt(self, goals, memory, actions, environment):
        raise NotImplementedError

    def parse_response(self, response):
        raise NotImplementedError
```

Then a specific implementation like this:

```python
class AgentFunctionCallingActionLanguage(AgentLanguage):
    def construct_prompt(self, goals, memory, actions, environment):
        # Constructs prompt using OpenAI's tool calling format
        ...

    def parse_response(self, response):
        # Extract tool name and arguments from function call
        ...
```

---

### 📌 Summary

| Concept         | Is it a language? | Is it a Python class? | Used for                               |
| --------------- | ----------------- | --------------------- | -------------------------------------- |
| `AgentLanguage` | ❌                 | ✅                     | Prompt construction + response parsing |




Let’s walk through how to build a simple `AgentLanguage` class tailored for **OpenAI’s function calling** interface. This class will:

1. 🧱 Construct prompts in the `messages` format.
2. 🧰 Attach tool definitions for function calling.
3. 🧠 Parse responses to extract tool calls cleanly.

---

## ✅ Step-by-Step: Build `AgentFunctionCallingLanguage`

### 📦 1. Define the Class Skeleton

```python
class AgentFunctionCallingLanguage:
    def __init__(self):
        pass
```

---

### 🏗️ 2. Construct the Prompt

We want to generate the `messages` list OpenAI expects:

```python
def construct_prompt(self, goals, memory, actions):
    system_message = {
        "role": "system",
        "content": self._format_goals(goals)
    }

    memory_messages = memory.get_messages()  # This could be a list of dicts with role/content
    return [system_message] + memory_messages
```

Helper to format goals:

```python
def _format_goals(self, goals):
    lines = ["You are an AI agent. Your goals:"]
    for goal in sorted(goals, key=lambda g: g.priority):
        lines.append(f"- {goal.name}: {goal.description}")
    return "\n".join(lines)
```

---

### 🧰 3. Return Tool Definitions

We need to extract the `tools` OpenAI uses for function calling:

```python
def get_tool_definitions(self, actions):
    tools = []
    for action in actions.get_actions():
        tools.append({
            "type": "function",
            "function": {
                "name": action.name,
                "description": action.description,
                "parameters": action.parameters
            }
        })
    return tools
```

---

### 🔍 4. Parse the Response

After calling the model, parse its tool selection:

```python
def parse_response(self, response):
    tool_call = response.choices[0].message.tool_calls[0]
    return {
        "tool": tool_call.function.name,
        "args": json.loads(tool_call.function.arguments)
    }
```

---

## 🧪 Example Usage with OpenAI

```python
language = AgentFunctionCallingLanguage()

prompt = language.construct_prompt(goals, memory, actions)
tools = language.get_tool_definitions(actions)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=prompt,
    tools=tools
)

invocation = language.parse_response(response)
print(invocation)
```

---

## ✅ Summary

| Component                | Purpose                                    |
| ------------------------ | ------------------------------------------ |
| `construct_prompt()`     | Formats the goals + memory into `messages` |
| `get_tool_definitions()` | Outputs tools for function calling         |
| `parse_response()`       | Reads the tool and arguments from response |


